# hypnogram

In [1]:
from __future__ import absolute_import, division, print_function 
#import os
#import re
#import sys
#import time

#sys.path.append(os.path.join('.', '..')) 
import utils
#import utils_DL
import utils_s160159 as u_s

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf


## Load data

In [2]:
data_dir = './../Data'
logs_path = './logs'
save_dir = './../Written work/Article/contents/'
SUBJECT_NO = 1
NUM_CLASSES = 6
BATCH_SIZE = 32

In [3]:
print("Loading subject %d..." %(SUBJECT_NO))
subjects_list = np.load(data_dir + '_dicts' + '/subject_' + str(SUBJECT_NO) + '_dict.npy').item()
IMAGE_SHAPE = subjects_list[0][0].shape

Loading subject 1...


In [4]:
def my_hyp(my_list, idx_scale,val_scale, file_name, savedir):
    def change(x):
        if x == 0: x_out = 0
        if x == 1: x_out = -1
        if x == 2: x_out = -2
        if x == 3: x_out = -3
        if x == 4: x_out = -4
        if x == 5: x_out = -5
        return(x_out)
    tmp_str = ''
    val_old = ''
    for idx, val in enumerate(my_list):
        val = change(val)
        idx *= idx_scale
        val *= val_scale

        if val_old == '':
            tmp_str += '(' + '{:.2f}'.format(idx) + ',' + '{:.2f}'.format(val) + ')--'
        else:
            tmp_str += '(' + '{:.2f}'.format(idx) + ',' + '{:.2f}'.format(old_val) + ')--' + \
            '(' + '{:.2f}'.format(idx) + ',' + '{:.2f}'.format(val) + ')--'

        val_old = val
    tmp_str = '\\draw[line width=0.05mm] ' + tmp_str[:-2] + ';'
    
    #
    text_file = open(savedir + file_name + '.tex', "w")
    text_file.write(tmp_str)
    text_file.close()
    print('Saved...')

In [5]:
my_hyp(my_list=subjects_list[1], 
       idx_scale=0.01,
       val_scale=0.1,
       file_name = 'hyp_class',
       savedir=save_dir)

Saved...


## Restoring the master model and  create per class sensitivity map

In [6]:
inputs_val = subjects_list[0]
targets_val = np.zeros((len(subjects_list[0]),NUM_CLASSES))
for ii in range(len(subjects_list[1])):
    targets_val[ii,subjects_list[1][ii]] = 1

In [7]:
tf.reset_default_graph()
model_path = './models/master/Version_4.0/'
with tf.Session() as sess:
    try:
        # restore model   
        #First let's load meta graph and restore weights
        saver = tf.train.import_meta_graph(model_path + 'fold_1.ckpt.meta')
        saver.restore(sess, tf.train.latest_checkpoint(model_path))
        graph = tf.get_default_graph()
        
        x_pl = graph.get_tensor_by_name(name='input_placeholder:0')
        y_pl = graph.get_tensor_by_name(name='target_placeholder:0')
        probs = graph.get_tensor_by_name('Softmax:0')
        prediction = tf.one_hot(tf.argmax(probs, axis=1), depth=NUM_CLASSES)
        
        #
        print('Evaluate Subject predict')
        val_pred = []
        _iter = 1
        #
        for x_batch, _ in utils.iterate_minibatches(batchsize=BATCH_SIZE, 
                                                          inputs=inputs_val, 
                                                          targets=targets_val, 
                                                          shuffle=False):
            _pred = sess.run(fetches=prediction,
                             feed_dict={x_pl: x_batch})
            # append prediction
            val_pred += [np.argmax(_pred[ii]) for ii in range(len(_pred))]
            
            print("\tminibatch: %d" %(_iter),end='\r')
            _iter += 1        
            
        # close session
        sess.close()
        print('')
        my_hyp(my_list=val_pred, 
               idx_scale=0.01,
               val_scale=0.1,
               file_name = 'hyp_master',
               savedir=save_dir)
        
        #
    except KeyboardInterrupt:
        pass

INFO:tensorflow:Restoring parameters from ./models/master/Version_4.0/fold_1.ckpt
Evaluate Subject predict
	minibatch: 58
Saved...


In [8]:
tf.reset_default_graph()
model_path = './models/rnn/Version_4.0/'
with tf.Session() as sess:
    try:
        # restore model   
        #First let's load meta graph and restore weights
        saver = tf.train.import_meta_graph(model_path + 'fold_1.ckpt.meta')
        saver.restore(sess, tf.train.latest_checkpoint(model_path))
        graph = tf.get_default_graph()
        
        x_pl = graph.get_tensor_by_name(name='input_placeholder:0')
        y_pl = graph.get_tensor_by_name(name='target_placeholder:0')
        probs = graph.get_tensor_by_name('Softmax:0')
        prediction = tf.one_hot(tf.argmax(probs, axis=1), depth=NUM_CLASSES)
        
        #
        print('Evaluate Subject predict')
        val_pred = []
        _iter = 1
        #
        for x_batch, _ in utils.iterate_minibatches(batchsize=BATCH_SIZE, 
                                                          inputs=inputs_val, 
                                                          targets=targets_val, 
                                                          shuffle=False):
            # LSTM has a static structure...
            if len(x_batch) != BATCH_SIZE: break
                
            _pred = sess.run(fetches=prediction,
                             feed_dict={x_pl: x_batch})
            # append prediction
            val_pred += [np.argmax(_pred[ii]) for ii in range(len(_pred))]
            
            print("\tminibatch: %d" %(_iter),end='\r')
            _iter += 1        
            
        # close session
        sess.close()
        print('')
        my_hyp(my_list=val_pred, 
               idx_scale=0.01,
               val_scale=0.1,
               file_name = 'hyp_rnn',
               savedir=save_dir)
        
        #
    except KeyboardInterrupt:
        pass
        


INFO:tensorflow:Restoring parameters from ./models/rnn/Version_4.0/fold_1.ckpt
Evaluate Subject predict
	minibatch: 56
Saved...
